In [19]:
import matplotlib.pyplot as plt
import requests
import asyncio
import nest_asyncio
import copy
import numpy as np
import os
import time
nest_asyncio.apply()  # Enable nested asyncio support

In [20]:
docker_url = "http://localhost:5000"

## With 3 servers

In [21]:
payload = {
    "N": 3,
    "schema": {
        "columns": ["Stud_id", "Stud_name", "Stud_marks"],
        "dtypes": ["Number", "String", "Number"],
    },
    "shards": [
        {"Stud_id_low": 0, "Shard_id": "sh1", "Shard_size": 4096},
        {"Stud_id_low": 4096, "Shard_id": "sh2", "Shard_size": 4096},
        {"Stud_id_low": 8192, "Shard_id": "sh3", "Shard_size": 4096},
    ],
    "servers": {
        "Server0": ["sh1", "sh2"],
        "Server1": ["sh2", "sh3"],
        "Server2": ["sh1", "sh3"],
    },
}
res = requests.post(f"{docker_url}/init", json=payload)
print(res.json())

{'message': 'Configured Database', 'status': 'success'}


In [24]:
num_requests = 10000


async def make_read_request():
    payload = {"Stud_id": {"low": 1000, "high": 8889}}
    res = requests.get(f"{docker_url}/read", json=payload)


async def make_write_request():
    payload = {
        "data": [
            {"Stud_id": 2255, "Stud_name": "GHI", "Stud_marks": 27},
            {"Stud_id": 3524, "Stud_name": "JKBFSFS", "Stud_marks": 56},
        ]
    }
    res = requests.post(f"{docker_url}/write", json=payload)


async def make_all_requests():
    tasks = [make_read_request() for _ in range(num_requests)]
    tasks.extend([make_write_request() for _ in range(num_requests)])
    await asyncio.gather(*tasks)

In [25]:
asyncio.get_event_loop().run_until_complete(make_all_requests())

## With 7 servers

In [ ]:
payload = {
    "n": 3,
    "servers": {
        "Server4": ["sh1", "sh2"],
        "Server[5]": ["sh2", "sh3"],
        "Server[6]": ["sh1", "sh3"],
        "Server[7]": ["sh1", "sh2"],
    },
}
res = requests.post(f"{docker_url}/add", json=payload)
print(res.json())